In [3]:
from model import DeiTForFewShot
import torch
from dataset import get_loaders
from utils import get_args_parser, generate_confusion_matrix, map_labels, to_device
from timm.utils import accuracy
from timm.scheduler import create_scheduler
import training
import json
import time
from accelerate import Accelerator

In [4]:
# Namespace(aa='rand-m9-mstd0.5-inc1', ada_lr=0.05, ada_steps=40, arch='dino_small_patch16', aug_prob=0.9, aug_types=['color', 'translation'], base_sources=['aircraft', 'cu_birds', 'dtd', 'fungi', 'ilsvrc_2012', 'omniglot', 'quickdraw', 'vgg_flower'], batch_size=1, cdfsl_domains=['EuroSAT', 'ISIC', 'CropDisease', 'ChestX'], checkpoint_key='teacher', clip_grad=None, color_jitter=0.4, cooldown_epochs=10, cutmix=0.0, cutmix_minmax=None, data_path='/datasets01/imagenet_full_size/061417/', dataset='places', decay_epochs=30, decay_rate=0.1, deploy='vanilla', device='cuda:0', dist_eval=False, dist_url='env://', distillation_alpha=0.5, distillation_tau=1.0, distillation_type='none', distributed=False, drop=0.0, drop_path=0.1, epochs=100, eval=True, fp16=True, ignore_bilevel_ontology=False, ignore_dag_ontology=False, ignore_hierarchy_probability=0.0, image_size=128, img_size=224, lr=5e-05, lr_noise=None, lr_noise_pct=0.67, lr_noise_std=1.0, max_log_weight=0.6931471805599453, max_num_query=10, max_support_set_size=500, max_support_size_contrib_per_class=100, max_ways_upper_bound=50, min_examples_in_class=0, min_log_weight=-0.6931471805599453, min_lr=1e-06, min_ways=5, mixup=0.0, mixup_mode='batch', mixup_prob=1.0, mixup_switch_prob=0.5, model_ema=False, model_ema_decay=0.99996, model_ema_force_cpu=False, momentum=0.9, nClsEpisode=8, nEpisode=2000, nQuery=15, nSupport=5, nValEpisode=120, n_shot=5, no_pretrain=False, num_adapters=1, num_classes=1000, num_query=None, num_support=None, num_ways=None, num_workers=10, opt='adamw', opt_betas=None, opt_eps=1e-08, output_dir='outputs/meta_train_places', patch_size=16, patience_epochs=10, pin_mem=True, pretrained_checkpoint_path='.', pretrained_weights='', recount=1, remode='pixel', repeated_aug=False, reprob=0.25, resplit=False, resume='', sched='cosine', seed=0, shuffle=True, smoothing=0.0, start_epoch=0, teacher_model='regnety_160', teacher_path='', test_n_way=5, test_sources=['traffic_sign', 'mscoco', 'ilsvrc_2012', 'omniglot', 'aircraft', 'cu_birds', 'dtd', 'quickdraw', 'fungi', 'vgg_flower'], test_transforms=['resize', 'center_crop', 'to_tensor', 'normalize'], train_interpolation='bicubic', train_transforms=['random_resized_crop', 'jitter', 'random_flip', 'to_tensor', 'normalize'], unused_params=False, val_sources=['aircraft', 'cu_birds', 'dtd', 'fungi', 'ilsvrc_2012', 'omniglot', 'quickdraw', 'vgg_flower'], warmup_epochs=5, warmup_lr=1e-06, weight_decay=0.05, world_size=1)

class Parser:
    def __init__(self):
        self.eval = False
        self.distributed = False
        self.dist_eval = False
        self.seed = 0
        self.pin_mem = True
        self.batch_size = 1
        self.num_workers = 10
        self.dataset = 'places_600'
        self.nSupport = 5
        self.img_size = 224
        self.nClsEpisode = 8
        self.nQuery =15
        self.nEpisode = 20
        self.device = 'cuda'
        self.output_dir = './output'
        # GPU
        self.deterministic = False
        self.device_ids = "3, 4"
        self.n_gpu = 2
        # Optimizer params
        self.lr = 5e-05
        self.momentum = 0.9
        self.decay_epochs = 30
        self.decay_rate = 0.1
        self.epochs = 2
        self.sched = 'cosine'
        self.min_lr = 1e-06
        self.warmup_lr=1e-06
        self.warmup_epochs=5
        self.cooldown_epochs=10
        
args = Parser()

In [5]:
# Get dataset 
data_loader_train, data_loader_val, class_to_label = get_loaders(args, 1, 0)

# Create model from pretrained backbone
model = DeiTForFewShot.from_pretrained("facebook/deit-base-distilled-patch16-224")
model.config.update({"id2label": class_to_label})

Some weights of the model checkpoint at facebook/deit-base-distilled-patch16-224 were not used when initializing DeiTForFewShot: ['cls_classifier.weight', 'cls_classifier.bias', 'distillation_classifier.weight', 'distillation_classifier.bias']
- This IS expected if you are initializing DeiTForFewShot from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DeiTForFewShot from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# Optimizers 
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

lr_scheduler , _ = create_scheduler(args, optimizer)

loss_function = torch.nn.CrossEntropyLoss()

In [7]:
accelerator = Accelerator()

model, optimizer, training_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, data_loader_train, lr_scheduler
)

In [ ]:
# Training 
print(f"Start training for {args.epochs} epochs")
start_time = time.time()
max_accuracy = 0


for epoch in range(args.epochs):
    train_stats = training.train_one_epoch(data_loader_train, model, loss_function, optimizer, epoch, device)
    
    lr_scheduler.step(epoch)
    
    evaluation_stats = training.evaluate(data_loader_val, model, loss_function, device, class_to_label)
    
    print(f"Accuracy on validation dataset: {evaluation_stats['acc']:.2f}% ± {evaluation_stats['confidence_interval']:.4f}%")
    
    max_accuracy = max(max_accuracy, evaluation_stats['acc'])
    print(f'Max accuracy: {max_accuracy:.2f}%')
    

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))


In [ ]:
metrics['epoch'] = -1
output_metrics = metrics.copy()
output_metrics.pop('y_target', None)
output_metrics.pop('y_pred', None)
with open(args.output_dir + "/log.txt", "a") as f:
    f.write(json.dumps(metrics) + "\n")

In [ ]:
generate_confusion_matrix(metrics['y_target'], metrics['y_pred'], list(class_to_label.keys()), './output')

In [10]:
a = "[1,2]"
list(a)

['[', '1', ',', '2', ']']